In [1]:
import json
import requests
import csv
import jsonlines
from flask import Flask, redirect, render_template, request, url_for
import sys
import numpy as np
from collections import defaultdict
import random
import math
import openai
import re
#csv.field_size_limit(sys.maxsize)

In [2]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [3]:
openai.api_key = ''

In [4]:
dev_reader = jsonlines.open(r'dev.jsonlines')
data = [datum for datum in dev_reader]

In [5]:
f = open('D:/Projects/OneIE/resource/valid_patterns_rams/event_role.json')
event_role= json.load(f)

In [8]:
dataset=data[0:20]

In [9]:
def call_gpt(messages, model="gpt-3.5-turbo-0613"):
    completions = openai.ChatCompletion.create(
        model=model,
        n=1,
        stop=None,
        temperature = 0.0,
        messages=messages)
    gpt_response = completions['choices'][0]['message']['content'].strip() 
    return gpt_response   

In [10]:
def merge_sentences(datum_sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in datum_sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph


In [11]:
def get_arguments(article, model="gpt-3.5-turbo-0613"):
    messages = [ 
        {
            "role": "system", 
            "content": """
                You are an extraction system that extracts the main characters of a news article.
                The main characters can be any organization, person or location that are heavily involved in the event described by the news article.
                The user will provide you with a news article to extract.
                Reply in the format '[character 1] [character 2]...'
            """
        },
        {
            "role": "user", "content": article
        } 
    ]
    arguments = call_gpt(messages)

In [12]:
def extract_events(sentence,event_role):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are an event extraction system. Your task is to extract events from news articles provided by the user. 
                An 'event' should contain one or more 'participants', which are the major participants in the event,
                and a 'trigger' is a verb that describes what kind of event happens between the participants. There are categories of events, and each trigger recognized is classified as one of the categories.
            """
        },
        {
            "role": "system", 
            "content": """ The user will provide a dictionary defining 'event category' as 'keys' and the 'types of participants' that can be involved in that event by 'values' of those keys.
                The format of the input dictionary will be like "['event type 1':['participant type 1','participant type 2', ...]], ['event type 2':['participant type 1','participant type 2', ...]], ...".
                Use this input to classify triggers and participants. if multiple participants are involved in an event you can create new roles to assign to the participants.
                Classify the triggers into one of the categories specified by the dictionary or create a new category. 
                Also classify the type of participants according to the user input , or create a new category.
            """
        },
        { "role": "user", "content": f"This is the input dictionary:{event_role}"},
        {
            "role": "system", 
            "content": """Now extract the triggers involved in the articles provided by the user and classify them into their event types.
                The triggers and participants should be human-readable. 
                Reply with each line being an event in the format:
                [event type 1 - trigger 1],[participant type 1 - participant 1], [participant type 2 - participant 2], ...
            """
        },
        {
            "role": "system", 
            "content": """ If an event contains multiple participants create an event containing roles for all participants
            """
        },
        { "role": "system", "name": "example_user", "content": "The article discussed how despite Donald J. Trump's promises to disrupt the political establishment, lobbyists and big donors were still present and active at the Republican National Convention. Lobbyists were seen mingling with Republican ambassadors, lawmakers, and executives, and even pitched a super PAC to party donors. Some who had previously opposed Trump's candidacy now saw opportunity in his rise."},
        { "role": "system", "name": "example_system", "content": "[contact:collaborate-mingling], [participant- lobbyists],[participant- Republican ambassadors],[participant- lawmakers],[participant- executives],[place-Republican National Convention];\n [contact:commitment promise express intent-pitched], [communicator- lobbyists],[recipient- party donors],[recipient- super PAC],[place-Republican National Convention];\n "},
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[conflict:attack - hurling], [attacker-fans],[target-fans],[instrument- bottles chairs and other objects],[place- Marseille]; \n [conflict:attack-fire] , [attacker-police],[target-fans],[instrument-tear gas],[place- Marseille];"},
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events


In [13]:
def extract_events1(sentence,event_role):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are an event extraction system. Your task is to extract events from news articles provided by the user. 
                An 'event' should contain one or more 'participants', which are the major participants in the event,
                and a 'trigger' is a verb that describes what kind of event happens between the participants. There are categories of events, and each trigger recognized is classified as one of the categories.
            """
        },
        {
            "role": "system", 
            "content": """ The user will provide an input defining the event categories and the types of participants that can be involved in that event.
                The format of the input will be like "[event type 1],[participant type 1],[participant type 2],...; [event type 2],[participant type 1],[participant type 2],...;".
                Use this input to classify triggers and participants into their types. 
                Classify the triggers into one of the categories specified by the input strictly, if none of them apply answer unknown. 
                Also classify the type of participants strictly according to the user input.
            """
        },
        { "role": "user", "content": f"This is the input dictionary:{event_role}"},
        {
            "role": "system", 
            "content": """Now extract the triggers involved in the articles provided by the user and classify them into their event types.
                The triggers and participants should be human-readable.
                participants should strictly be exact characters extracted from the article. 
                Reply with each line being an event in the format:
                [event type 1 - trigger 1],[participant type 1 - participant 1],[participant type 2 - participant 2], ...
            """
        },
        { "role": "system", "name": "example_user", "content": "The article discussed how despite Donald J. Trump's promises to disrupt the political establishment, lobbyists and big donors were still present and active at the Republican National Convention. Lobbyists were seen mingling with Republican ambassadors, lawmakers, and executives, and even pitched a super PAC to party donors. Some who had previously opposed Trump's candidacy now saw opportunity in his rise."},
        { "role": "system", "name": "example_system", "content": "[contact:collaborate-mingling], [participant- lobbyists],[participant- Republican ambassadors],[participant- lawmakers],[participant- executives],[place-Republican National Convention];\n [contact:commitment promise express intent-pitched], [communicator- lobbyists],[recipient- party donors],[recipient- super PAC],[place-Republican National Convention];\n "},
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[conflict:attack - hurling], [attacker-fans],[target-fans],[instrument- bottles chairs and other objects],[place- Marseille]; \n [conflict:attack-fire] , [attacker-police],[target-fans],[instrument-tear gas],[place- Marseille];"},
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events


In [14]:
def extract(sentence):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art event extraction system. Your task is to extract events from news articles provided by the user. 
                An event represents an action or occurrence described in the text. Events are always associated with specific triggers and involve various entities.
                Entities are the elements or participants involved in an event. These can be people, organizations, locations, dates, or any other named entities that play a role in the event described in the text. 
                Each recognized entity plays a role in an event. 
                Roles, also known as event roles or thematic roles, are specific functions that entities play in relation to an event. These roles define the roles or positions that entities hold with respect to the action described by the trigger.
            """
        },
        { 
            "role": "system", 
            "content": """
                The user will provide an input containing news articles. You have to extract events from the text step wise.
                First recognize the triggers in the text.
                Then classify these triggers into the type of event it triggers.
                Then identify the major entities involved in the event.
                Finally assign roles to these entities. 
                An event cannot be recognized without first extracting a word from the text that indicates the occurence of that event.
                Each event should have a trigger and roles assigned to the entities involved in the event.
                Reply with each line being an event in the format:
                [event type 1 - trigger 1],[role type 1 - entity 1], [role type 2 - entity 2], ...
            """
        },
        { "role": "system", "name": "example_user", "content": "The article discussed how despite Donald J. Trump's promises to disrupt the political establishment, lobbyists and big donors were still present and active at the Republican National Convention. Lobbyists were seen mingling with Republican ambassadors, lawmakers, and executives, and even pitched a super PAC to party donors. Some who had previously opposed Trump's candidacy now saw opportunity in his rise."},
        { "role": "system", "name": "example_system", "content": "[contact collaborate-mingling], [participant- lobbyists],[participant- Republican ambassadors],[participant- lawmakers],[participant- executives],[place-Republican National Convention];\n [contact commitment promise express intent - pitched], [communicator- lobbyists],[recipient- party donors],[recipient- super PAC],[place-Republican National Convention];\n "},
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[conflict attack - hurling], [attacker-fans],[target-fans],[instrument- bottles chairs and other objects],[place- Marseille]; \n [conflict attack - fire] , [attacker-police],[target-fans],[instrument-tear gas],[place- Marseille];"},
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events

In [35]:
def get_entities(sentence,events):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art named entity recognition model. 
                Entities are the elements or participants involved in an event.
                These can be people, organizations, locations, dates, or any other named entities that play a role in the event described in the text. 
                Your task is to extract the major entities involved in the events described by the user input.
            """
        },
        { "role": "user", "content": f"This is the input containing the major events in each article:{events}"},
        { 
            "role": "system", 
            "content": """
                The user will provide articles as input.
                Now your task is to identify the major entities from these articles that are heavily involved in the recognized events.
                Reply with each line being an event in the format:
                [entity 1],[entity 2];\n...
            """
        },
        { "role": "system", "name": "example_user", "content": "The article discussed how despite Donald J. Trump's promises to disrupt the political establishment, lobbyists and big donors were still present and active at the Republican National Convention. Lobbyists were seen mingling with Republican ambassadors, lawmakers, and executives, and even pitched a super PAC to party donors. Some who had previously opposed Trump's candidacy now saw opportunity in his rise."},
        { "role": "system", "name": "example_system", "content": "[lobbyists],[Republican ambassadors],[lawmakers],[executives],[Republican National Convention],[party donors],[super PAC],[Donald J. Trump];\n "},
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[fans],[bottles],[chairs],[other objects],[Marseille],[police],[tear gas][Euro 2016],[rival supporters],[French port city],[stadium],[England supporters];\n"},
        { "role": "user", "content": f"This is the input containing all articles:{sentence}"},
    ]
    events = call_gpt(messages)
    return events

In [33]:
def extract_classes(sentence):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art event extraction system. Your task is to extract only the top two major events from news articles provided by the user. 
                A 'trigger' is a verb that describes what kind of event happens between the participants.
                All events have a trigger word in the input article given by user.
                There are categories of events, and each trigger recognized is classified as one of the categories.
            """
        },
        { 
            "role": "system", 
            "content": """
                Please extract the class of events and the trigger word triggering that event from user provided sentence.
                From each article at most two events have to be extracted. The number of events extracted should not exceed two. 
                These three events should be the major events that the article addresses.
                The event types and triggers should be human-readable.
                Reply with each line being an event in the format:
                [event type 1 - trigger 1]; \n [event type 2 - trigger 2];\n...
            """
        },
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[conflict:attack - hurling];\n[conflict:violence - violent]"},
        { "role": "system", "name": "example_user", "content": "The article discussed the recent events of two Democratic operatives, Robert Creamer and Scott Foval, losing their jobs after a leaked video revealed their controversial strategies for winning elections. Creamer, who is married to Rep. Jan Schakowsky, announced that he was stepping back from his work for the unified Democratic campaign for Hillary Clinton, while Foval was laid off by Americans United for Change."},
        { "role": "system", "name": "example_system", "content": "[personnel:end position - losing];\n [government:controversy - controversial];\n"},        
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events

In [17]:
def classes(sentence,cl):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art event extraction system. Your task is to identify major entities and assign roles that the identified entities play in an event. 
                Entities are the elements or participants involved in an event. These can be people, organizations, locations, dates, or any other named entities that play a role in the event described in the text. 
                Each recognized entity plays a role in an event. 
                Roles, also known as event roles or thematic roles, are specific functions that entities play in relation to an event. These roles define the roles or positions that entities hold with respect to the action described by the trigger.
                The user will provide an input defining the events and the trigger that indicates occurrence of that event. 
                The format of the input will be like:
                [event type 1 - trigger1]; \n [event type 2 - trigger 2];\n...
                Use this input to identify the event occuring in the article and the triggers that indicate the occurence of those events.
            """
        },
        { "role": "user", "content": f"This is the input:{cl}"},
        { "role": "system", "content": """            
                Please extract the major entities and assign roles that the identified entities play in an event. 
                Reply with each line being an event in the format:
                [event type 1 - trigger 1],[role 1 - entity 1],[role 2 - entity 2], ...
         """
        },
        { "role": "system", "name": "example_user", "content": "The article discussed how despite Donald J. Trump's promises to disrupt the political establishment, lobbyists and big donors were still present and active at the Republican National Convention. Lobbyists were seen mingling with Republican ambassadors, lawmakers, and executives, and even pitched a super PAC to party donors. Some who had previously opposed Trump's candidacy now saw opportunity in his rise."},
        { "role": "system", "name": "example_system", "content": "[contact:collaborate-mingling], [participant- lobbyists],[participant- Republican ambassadors],[participant- lawmakers],[participant- executives],[place-Republican National Convention];\n [contact:commitment promise express intent-pitched], [communicator- lobbyists],[recipient- party donors],[recipient- super PAC],[place-Republican National Convention];\n "},
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[conflict:attack - hurling], [attacker-fans],[target-fans],[instrument- bottles chairs and other objects],[place- Marseille]; \n [conflict:attack-fire] , [attacker-police],[target-fans],[instrument-tear gas],[place- Marseille];"},
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events

In [18]:
def all_merged(sentence,cl,ar):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art event extraction system. Your task is to assign roles that the identified entities play in an event. 
                The major identified entities will be given as user input.
                the format of input will be "[entity 1],[entity 2],..."
                """
        },
        { "role": "user", "content": f"This is the input containing recognized entities:{ar}"},
        { "role": "system", "content": """            
                Roles, also known as event roles or thematic roles, are specific functions that entities play in relation to an event. 
                These roles define the roles or positions that entities hold with respect to the action described by the trigger.
                The user will provide an input defining the events and the trigger that indicates occurrence of that event. 
                The format of the input will be like:
                [event type 1 - trigger1]; \n [event type 2 - trigger 2];\n...
                Use this input to identify the event occuring in the article and the triggers that indicate the occurence of those events.
         """
        },
        { "role": "user", "content": f"This is the input:{cl}"},
        { "role": "system", "content": """            
                Please assign roles that the identified major entities play in an event. 
                Reply with each line being an event in the format:
                [event type 1 - trigger 1],[role 1 - entity 1],[role 2 - entity 2], ...
         """
        },
        {
            "role": "system", 
            "content": """ If an event contains multiple participants create an event containing roles for all participants.
            """
        },
        { "role": "system", "name": "example_user", "content": "The article discussed how despite Donald J. Trump's promises to disrupt the political establishment, lobbyists and big donors were still present and active at the Republican National Convention. Lobbyists were seen mingling with Republican ambassadors, lawmakers, and executives, and even pitched a super PAC to party donors. Some who had previously opposed Trump's candidacy now saw opportunity in his rise."},
        { "role": "system", "name": "example_system", "content": "[contact:collaborate-mingling], [participant- lobbyists],[participant- Republican ambassadors],[participant- lawmakers],[participant- executives],[place-Republican National Convention];\n [contact:commitment promise express intent-pitched], [communicator- lobbyists],[recipient- party donors],[recipient- super PAC],[place-Republican National Convention];\n "},
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[conflict:attack - hurling], [attacker-fans],[target-fans],[instrument- bottles chairs and other objects],[place- Marseille]; \n [conflict:attack-fire] , [attacker-police],[target-fans],[instrument-tear gas],[place- Marseille];"},
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events

In [48]:
def merged(sentence,cl,ar):
    messages = [
        { 
            "role": "system", 
            "content": """
                Your task is to assign roles that the entities play in an event.
                Major entities and events in the article will be provided by the user.
                Each article has at most two major events.
            """
        },
        { "role": "system", "content": """            
                The user will provide an input defining the events and the trigger that indicates occurrence of that event. Do not extract any events other than the ones defined in this input.
                These are the only major events occuring in the article. Strictly use only these events for assigning roles to participants.
                The format of the input will be like:
                [event type 1 - trigger1]; \n [event type 2 - trigger 2];\n...
         """
        },
        { "role": "user", "content": f"This is the input containing event types and triggers:{cl}"},
        { 
            "role": "system", 
            "content": """
                The major identified entities will be given as user input.
                These are the only major entities that can be involved in the events occuring in the article.
                the format of input will be "[entity 1],[entity 2],..."
                """
        },
        { "role": "user", "content": f"This is the input containing recognized entities in an article:{ar}"},
        { "role": "system", "content": """            
                Strictly assign roles only to those participants that are involved in the event.
                Reply with each line being an event in the format:
                [event type 1 - trigger 1],[role 1 - entity 1],[role 2 - entity 2], ...
         """
        },
        # {
        #     "role": "system", 
        #     "content": """ If an event contains multiple participants create an event containing roles for all participants.
        #     """
        # },
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "[conflict:attack - hurling],[attacker - fans],[victim - rival fans],[place - Euro 2016 match];\n[conflict:violence - violent],[participant - men],[victim-another person],[action-kicking and stomping];"},
        { "role": "system", "name": "example_user", "content": "The article discussed the recent events of two Democratic operatives, Robert Creamer and Scott Foval, losing their jobs after a leaked video revealed their controversial strategies for winning elections. Creamer, who is married to Rep. Jan Schakowsky, announced that he was stepping back from his work for the unified Democratic campaign for Hillary Clinton, while Foval was laid off by Americans United for Change."},
        { "role": "system", "name": "example_system", "content": "[personnel:end position - losing],[employee - Robert Creamer],[employee - Scott Foval],[place of employment - the unified Democratic campaign];\n [government:controversy - controversial],[participant - Robert Creamer],[participant - Scott Foval],[source - video],[action - controversial strategies for winning elections];\n"},   
        { "role": "user", "content": f"This is the article:{sentence}"},
    ]
    events = call_gpt(messages)
    return events

In [19]:
def assign_classes(sentence,classes):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are an event extraction system. Your task is to extract events from news articles provided by the user. 
                An 'event' should contain one or more 'participants', which are the major participants in the event,
                and a 'trigger' is a verb that describes what kind of event happens between the participants.
                All events have a trigger word in the input article given by user.
                  There are categories of events, and each trigger recognized is classified as one of the categories.
            """
        },
        { 
            "role": "system", 
            "content": """
                The user will provide an input defining the event categories and the types of participants that can be involved in that event.
                The format of the input will be like "[event type 1],[participant type 1],[participant type 2],...; [event type 2],[participant type 1],[participant type 2],...;".
                Use this input to classify triggers and participants into their types. If none of them apply create a new event category and assign roles to all participants compulsorily.
            """
        },
        { "role": "user", "content": f"This is the input defining event and participant types:{classes}"},
        {
            "role": "system", 
            "content": """Now extract the triggers involved in the articles provided by the user and classify them into their event types.
                Strictly return a trigger for each event.
                The triggers and participants should be human-readable.
                participants should strictly be exact characters extracted from the article. 
                Reply with each line being an event in the format:
                ['event type 1' - 'trigger 1'],[participant type 1 - participant 1],[participant type 2 - participant 2], ...
            """
        },
        {
            "role": "system", 
            "content": """ If an event contains multiple participants create an event containing roles for all participants
            """
        },
        { "role": "system", "name": "example_user", "content": "The article discussed how despite Donald J. Trump's promises to disrupt the political establishment, lobbyists and big donors were still present and active at the Republican National Convention. Lobbyists were seen mingling with Republican ambassadors, lawmakers, and executives, and even pitched a super PAC to party donors. Some who had previously opposed Trump's candidacy now saw opportunity in his rise."},
        { "role": "system", "name": "example_system", "content": ""},
        { "role": "system", "name": "example_user", "content": "The fixture anticipated by fans as the marquee game of the second day of Euro 2016 ended in a 1 - 1 tie . In chaotic scenes involving hundreds of fans throngs of rival supporters rushed at one another hurling bottles chairs and other objects and forcing police in riot gear to fire tear gas in response . Fans skirmish ahead of the match in the French port city of Marseille on Saturday . Distressing footage emerged of men kicking and stomping on another person lying on the street in broad daylight as the rivalry turned violent prior to kick - off . Then at the end of the game Russian fans charged at the section of the stadium containing England supporters and more chaos ensued."},
        { "role": "system", "name": "example_system", "content": "['conflict:attack' - hurling], [attacker-fans],[target-fans],[instrument- bottles chairs and other objects],[place- Marseille]; \n ['conflict:attack'-'fire'] , [attacker-police],[target-fans],[instrument-tear gas],[place- Marseille];"},
        { "role": "user", "content": sentence}
    ]
    events = call_gpt(messages)
    return events

In [20]:
def summarize_sentence(article, arguments, model="gpt-3.5-turbo-0613"):
    messages = [ 
        {
            "role": "system", 
            "content": """
                You are an summarization system that summarizes the events that happened between the main characters of a news article.
                The user will provide you with a list of main characters and a news article to summarize.
                Try to summarize the article with no more than three sentences. 
                Reply starts with 'The article discussed ...'
            """
        },
        {
            "role": "user", "content": "Main Characters:\n{arguments} \n\n\n Article: {article}".format(arguments=arguments, article=article)
        } 
    ]
    sentence = call_gpt(messages)
    return sentence

In [22]:
def strip_sentence(sentence):
    if sentence.startswith('The article discussed how'):
        stripped_sentence = sentence.replace('The article discussed how', '').strip()
    elif sentence.startswith('The article discussed'):
        stripped_sentence = sentence.replace('The article discussed', '').strip()
    else:
        print("!!!")
    return stripped_sentence

In [21]:
# RAMS
saved_dataset = []
for datum in dataset:
    saved_datum = {}
    article = merge_sentences(datum['sentences'])
    arguments = get_arguments(article)
  #  sentence = summarize_sentence(article, arguments)
  #  print(sentence)
    # saved_datum['content'] = datum['sentences']
    # saved_datum['url'] = datum['source_url']
    # saved_datum['summary'] = sentence
    saved_dataset.append(arguments)
save_json(saved_dataset, r'D:/Projects/Test/summarized/args.json')

In [51]:
AllTheNews_summarized = json.load(open(r'D:/Projects/Test/All the News/data/summarized/summary.json'))
# cl = json.load(open(r'D:/Projects/Test/All the News/events/classes.json'))
# ar = json.load(open(r'D:/Projects/Test/All the News/events/cl_ents.json'))
res_events = []
error_datum = []
for index, datum in enumerate(AllTheNews_summarized):
    print('{}/{}'.format(index, len(AllTheNews_summarized)))
    sentence = strip_sentence(datum['summary'])
    cl = extract_classes(sentence)
    ent = get_entities(sentence,cl)
    events = merged(sentence,cl,ent)
    datum['events'] = events
    res_events.append(datum)
save_json(res_events, r'D:/Projects/Test/All the News/events/merged_reduced.json')

0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


In [37]:
AllTheNews_summarized = json.load(open(r'D:/Projects/Test/All the News/data/summarized/summary.json'))
cl = json.load(open(r'D:/Projects/Test/All the News/events/classes.json'))
res_events = []
error_datum = []
for index, datum in enumerate(AllTheNews_summarized):
    print('{}/{}'.format(index, len(AllTheNews_summarized)))
    sentence = strip_sentence(datum['summary'])
    events = get_entities(sentence,cl)
    datum['events'] = events
    res_events.append(events)
save_json(res_events, r'D:/Projects/Test/All the News/events/cl_ents.json')

0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


'[conflict:cyberattack - targeting];\n [government:political appointment - choice]'

In [23]:
AllTheNews_summarized = json.load(open(r'D:/Projects/Test/All the News/data/summarized/summary.json'))
cl = json.load(open(r'D:/Projects/Test/All the News/events/events.json'))
res_events = []
error_datum = []
for index, datum in enumerate(AllTheNews_summarized):
    print('{}/{}'.format(index, len(AllTheNews_summarized)))
    sentence = strip_sentence(datum['summary'])
    events = classes(sentence,cl)
    datum['events'] = events
    res_events.append(datum)
save_json(res_events, r'D:/Projects/Test/All the News/events/events_cl.json')

0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


In [25]:
AllTheNews_summarized = json.load(open(r'D:/Projects/Test/All the News/data/summarized/summary.json'))
res_events = []
error_datum = []
for index, datum in enumerate(AllTheNews_summarized):
    print('{}/{}'.format(index, len(AllTheNews_summarized)))
    sentence = strip_sentence(datum['summary'])
    classes = extract(sentence)
    #events = extract_events1(sentence,classes)
    datum['events'] = classes
    res_events.append(datum)
save_json(res_events, r'D:/Projects/Test/All the News/events/new2.json')

0/20
1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20


In [ ]:
AllTheNews_summarized = json.load(open(r'D:/Projects/Test/All the News/data/summarized/summary.json'))
class_types = json.load(open(r'D:/Projects/Test/All the News/events/classes.json'))
res_events = []
error_datum = []
for index, datum in enumerate(AllTheNews_summarized):
    print('{}/{}'.format(index, len(AllTheNews_summarized)))
    sentence = strip_sentence(datum['summary'])
    #classes = extract_classes(sentence)
    events = assign_classes(sentence,class_types)
    datum['events'] = events
    res_events.append(datum)
save_json(res_events, r'D:/Projects/Test/All the News/events/cl.json')

In [ ]:
RAMS_summarized = json.load(open(r'D:/Projects/Test/summarized/dev.json'))
res_events = []
error_datum = []
for index, datum in enumerate(RAMS_summarized):
    try:
        print('{}/{}'.format(index, len(RAMS_summarized)))
        sentence = strip_sentence(datum['summary'])
        events = extract_classes(sentence)
        datum['events'] = events
        res_events.append(datum)
    except:
        error_datum.append(datum)
save_json(res_events, r'D:/Projects/Test/events/classes.json')

In [ ]:
from string import punctuation
def post_process_events(dataset):
    for index, datum in enumerate(dataset):
        datum['doc_id'] = index
        datum['events_raw'] = datum['events']
        events_str = datum['events'].split('\n')
        events = []
        for event_str in events_str:
            arguments=[]
            event_str=event_str[1:-1]
            components = event_str.split(',')
            event_type_raw = components[0].split('-')
            event_type=event_type_raw[0].strip()
            trigger_raw = event_type_raw[1].split(',')
            trigger = trigger_raw[0].strip()
            trigger = trigger[0:-1]
            arguments_raw = [arg.strip().strip(punctuation) for arg in components[1:]]
            for dat in arguments_raw:
                temp = dat.split(',')
                for arg in temp:
                    arg_raw=arg.split('-')
                    arg_type=arg_raw[0].strip()
                    args = arg_raw[1:]
                    args_final=' '.join([str(elem) for elem in args]).strip()
                    arguments.append({arg_type:args_final})
            events.append({'event_type':event_type,'trigger': trigger, 'arguments': arguments})
        datum['events'] = events
    return dataset

dataset = json.load(open(r'D:/Projects/Test/events/dev4.json'))
processed_dataset = post_process_events(dataset)
save_json(processed_dataset, r'D:/Projects/Test/result/gpt_events_dev2.json')


In [50]:
from string import punctuation
def post_process_events(dataset):
    for index, datum in enumerate(dataset):
        datum['doc_id'] = index
        datum['events_raw'] = datum['events']
        events_str = datum['events'].split('\n')
        events = []
        for event_str in events_str:
            arguments=[]
            event_str = event_str.strip()
            components = event_str.split('],')
            event_type_raw = components[0].split('-')
            event_type=event_type_raw[0].strip()
            event_type = event_type[1:]
            trigger = event_type_raw[1:]
            trigger=' '.join([str(elem) for elem in trigger]).strip().strip(punctuation)
            if(trigger==""):
                trigger=event_type
            arguments_raw = [arg.strip().strip(punctuation) for arg in components[1:]]
            for dat in arguments_raw:
                dat=re.sub(",","",dat)
                temp = dat.split(',')
                for arg in temp:
                    arg=re.sub(",","",arg)
                    arg_raw=arg.split('-')
                    arg_type=arg_raw[0].strip()
                    args = arg_raw[1:]
                    args_final=' '.join([str(elem) for elem in args]).strip().strip(punctuation)
                    arguments.append({arg_type:args_final})
            events.append({'event_type':event_type,'trigger': trigger, 'arguments': arguments})
        datum['events'] = events
    return dataset

dataset = json.load(open(r'D:/Projects/Test/All the News/events/merged_reduced.json'))
processed_dataset = post_process_events(dataset)
save_json(processed_dataset, r'D:/Projects/Test/All the News/result/merged_reduced.json')


In [ ]:
from string import punctuation
def post_process_events(dataset):
    for index, datum in enumerate(dataset):
        datum['doc_id'] = index
        datum['events_raw'] = datum['events']
        events_str = datum['events'].split('\n')
        events = []
        for event_str in events_str:
            arguments=[]
            event_str=event_str[1:-1]
            components = event_str.split(',')
            event_type_raw = components[0].split('-')
            event_type=event_type_raw[0].strip().strip(punctuation)
            trigger_raw = ''.join([str(elem) for elem in event_type_raw[1:]])
            #print(trigger_raw)
            trigger_raw = trigger_raw[1:].split(',')
            trigger = trigger_raw[0].strip()
           # trigger = trigger[0:-1]
            trigger = event_type_raw[1].strip().strip(punctuation)
            # a = ' '.join([str(elem) for elem in components[1:]]).strip().strip(punctuation)
            # # print(a)
            arguments_raw = [arg.strip().strip(punctuation) for arg in components[1:]]
            #print(components[1:])
            for dat in arguments_raw:
                # print(dat)
                # temp = dat.split(',')
                # for arg in temp:
                arg_raw=dat.split('-')
                arg_type=arg_raw[0].strip().strip(punctuation)
                args = arg_raw[1:]
                args_final=' '.join([str(elem) for elem in args]).strip().strip(punctuation)
                arguments.append({arg_type:args_final})
            events.append({'event_type':event_type,'trigger': trigger, 'arguments': arguments})
        datum['events'] = events
    return dataset

dataset = json.load(open(r'D:/Projects/Test/All the News/events/event_classes1.json'))
processed_dataset = post_process_events(dataset)
save_json(processed_dataset, r'D:/Projects/Test/All the News/result/atn_withclass.json')
